### Transportation problem
$$
\begin{array}{rl}
    \min & 10 x_{11} + 12 x_{12} + 14 x_{13} + 11 x_{21} + 12 x_{22} + 13 x_{23}  \\
      \mbox{Subject to} & x_{11} + x_{12} + x_{13} \, \le \,   100 \nonumber \\
  & x_{21} + x_{22} + x_{23} \, \le \,   200 \nonumber \\
  & x_{11} + x_{21} \, \ge \,   50 \nonumber \\
  &  x_{12} + x_{22} \, \ge \,   100 \nonumber \\
  & x_{13} + x_{23} \, \ge \,   150 \nonumber \\
  & x \ge 0
\end{array}
$$

In [2]:
import numpy as np
import cvxpy as cp
import scipy

In [3]:
m = 2
n = 3
x = cp.Variable((m,n))

In [176]:
obj = cp.Minimize(10*x[0,0] + 12*x[0,1] + 14*x[0,2] + 11*x[1,0] + 12*x[1,1] + 13*x[1,2])

In [177]:
constraints = [
    x[0,0] + x[0,1] + x[0,2] <= 100,
    x[1,0] + x[1,1] + x[1,2] <= 200,
    x[0,0] + x[1,0] >= 50,
    x[0,1] + x[1,1] >= 100,
    x[0,2] + x[1,2] >= 150,
    x >= 0
]

In [178]:
prob = cp.Problem(obj,constraints)

In [182]:
prob.solve()
if (prob.status == 'optimal'):
    print('value = ' + str(prob.value))
    print('x = ' + str(x.value))

value = 3650.000000025025
x = [[4.99999995e+01 4.99999997e+01 7.24977381e-07]
 [4.21322203e-07 5.00000003e+01 1.49999999e+02]]


In [183]:
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('value = ' + str(prob.value))
    print('x = ' + str(x.value))

value = 3650.0
x = [[ 50.  50.   0.]
 [  0.  50. 150.]]


### Investment problem
$$ 
\begin{array}{rrcl}
  \max_{x,y} & y_3\\
  \mbox{Subject to} & x_a + x_b + y_0  & = &  100\\
  & 0.1x_a + 0.2x_b - x_c + 1.02y_0 - y_1 & = & 0\\
  & 1.1 x_b + 1.02 y_1  - y_2  & = &   0\\
  & 1.3 x_a + 1.5 x_c + 1.02 y_2 - y_3  & = & 0\\
  & x, y & \ge & 0
\end{array}
$$

In [185]:
x = cp.Variable(3)
y = cp.Variable(4)

In [186]:
obj = cp.Maximize(y[3])

In [187]:
constraints = [
    x[0] + x[1] + y[0] <= 100,
    0.1*x[0] + 0.2*x[1] + 1.02*y[0] == x[2] + y[1],
    1.1*x[1]  + 1.02*y[1] == y[2],
    1.3*x[0] + 1.5*x[2] + 1.02*y[2] == y[3],
    x >= 0,
    y >= 0
]

In [188]:
prob = cp.Problem(obj,constraints)

In [189]:
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('value = ' + str(prob.value))
    print('x = ' + str(x.value))
    print('y = ' + str(y.value))

value = 153.0
x = [  0.   0. 102.]
y = [100.   0.   0. 153.]


### Linear programming using native scipy function

scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='interior-point', callback=None, options=None, x0=None)

Solves
$$
\begin{array}{rl}
\mbox{min} & c^\top x\\
\mbox{s.t.} & A_{ub} x \leq b_{ub}\\
& A_{eq} x = b_{eq}\\
& \ell \leq x \leq u
\end{array}
$$


In [121]:
# Solve cash flow problem using this function
from scipy.optimize import linprog

numx = 3
numy = 4
numvars = 7
T = 4

# objective vector
c = np.zeros(numvars)
c[numvars-1] = -1 # maximizing y[3] is the same as minimizing -y[3]

# constraint matrix
Aeq = np.zeros((T,numvars))
beq = np.zeros(T)
# first constraint
Aeq[0][0] = 1
Aeq[0][1] = 1
Aeq[0][numx] = 1
beq[0] = 100
# second constraint
Aeq[1][0] = 0.1
Aeq[1][1] = 0.2
Aeq[1][2] = -1
Aeq[1][numx] = 1.02
Aeq[1][numx+1] = -1
# third constraint
Aeq[2][1] = 1.1
Aeq[2][numx+1] = 1.02
Aeq[2][numx+2] = -1
# fourth constraint
Aeq[3][0] = 1.3
Aeq[3][2] = 1.5
Aeq[3][numx+2] = 1.02
Aeq[3][numx+3] = -1

bounds = (0,np.Inf)

res = linprog(c,A_eq=Aeq,b_eq=beq,bounds=bounds,method='simplex')
x = res.x[0:numx]
y = res.x[numx:numvars]
print('x = ' + str(x))
print('y = ' + str(y))

x = [  0.   0. 102.]
y = [100.   0.   0. 153.]


### Feed blending problem

The LP for this problem is as follows:
$$
\begin{array}{rl}
\min_x & 200x_o + 150x_c + 100x_a + 75x_h,\\
\mbox{s.t.} & 60x_o + 80x_c + 30x_a + 40x_h  \geq  60\\
& 50x_o + 70x_c + 40x_a + 100x_h \leq  60,\\
& 90x_o + 30x_c + 60x_a + 80x_h  \geq  60,\\
& x_o + x_c + x_a + x_h = 1,\\
& x \geq 0
\end{array}
$$

In [9]:
xo = cp.Variable(1)
xc = cp.Variable(1)
xa = cp.Variable(1)
xh = cp.Variable(1)

In [13]:
obj = cp.Minimize(200*xo + 150*xc + 100*xa + 75*xh)
constraints = [
    60*xo + 80*xc + 30*xa + 40*xh >= 60,
    50*xo + 70*xc + 40*xa + 100*xh <=  60,
    90*xo + 30*xc + 60*xa + 80*xc >=  60,
     xo + xc + xa + xh == 1,
    xo >= 0, xc >=0, xa >= 0, xh >= 0
]

In [15]:
prob = cp.Problem(obj,constraints)
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('cost = ' + str(prob.value))
    print('x_o = ' + str(xo.value))
    print('x_c = ' + str(xc.value))
    print('x_a = ' + str(xa.value))
    print('x_h = ' + str(xh.value))

cost = 128.7037037037037
x_o = [0.]
x_c = [0.59259259]
x_a = [0.37037037]
x_h = [0.03703704]


### Nurse scheduling problem

The LP here is as follows:
$$
\begin{array}{rl}
  \min_x & \sum_{j=0}^6 x_j,\\ 
  \mbox{s.t.} & \sum_{k=0}^4 x_{(j-k) \text{ mod } 6} \geq d_j, \qquad j =
                0, \ldots, 6,\\ 
  & x \geq 0.
\end{array}
$$


In [33]:
# Data
d = np.array([110, 100,  90,  90, 100, 115, 120])
T = 7

x = cp.Variable(T)

In [34]:
obj = cp.Minimize(sum(x))
constraints = []
for j in range(T):
    constraints += [sum([x[(j-k)%T] for k in range(T-2)]) >= d[j]]
constraints += [x >= 0]

In [35]:
prob = cp.Problem(obj,constraints)
prob.solve(solver=cp.GLPK)
if (prob.status == 'optimal'):
    print('number of nurses = ' + str(prob.value))
    print('x = ' + str(x.value))

number of nurses = 145.0
x = [10. 15. 20. 25. 30. 25. 20.]


### Inventory problem
In the following example we have four time periods
and per-unit production costs and capacities are given as
per the table below. Additionally, units product of the product at each
level of completion can be stored as inventory, at a per-unit cost and
also subject to capacities. The problem is to establish a production and inventory plan that delivers all demands at minimum cost.

In [162]:
# data

c = np.array([50,4000,30,22]) # production costs
u = np.array([100,50,54,90])  # production capacity
h = np.array([150,240,45]) # inventory cost
w = np.array([170,10,44]) # inventory capacity
d = np.array([70,50,40,90]) # demand

T = len(c)

In [163]:
x = cp.Variable(T) # production 
s = cp.Variable(T-1) # storage/inventory

In [164]:
obj_expr = []
for t in range(T):
    obj_expr += [c[t]*x[t]]
for t in range(T-1):
    obj_expr += [h[t]*s[t]]
obj = cp.Minimize(sum(obj_expr))

In [165]:
constraints = [
    x >= 0, s >= 0, x <= u, s <= w,
    x[0] == d[0] + s[0],
    x[1] + s[0] == d[1] + s[1],
    x[2] + s[1] == d[2] + s[2],
    s[2] + x[3] == d[3]
]

In [166]:
prob = cp.Problem(obj,constraints)

In [168]:
prob.solve(solver=cp.GLPK)

if (prob.status == 'optimal'):
    print('obj = ' + str(prob.value))
    print('x = ' + str(x.value))
    print('s = ' + str(s.value))

obj = 92680.0
x = [100.  20.  40.  90.]
s = [30.  0.  0.]


### Inventory control with back orders

Let $b_t$ denote the amount back ordered at time $t$. Then the new dynamics are given by 
$$
s_{t-1} + x_t + b_t = d_t + s_t + b_{t-1}
$$

In [169]:
x = cp.Variable(T) # production 
s = cp.Variable(T-1) # storage/inventory
b = cp.Variable(T-1) # back ordered quantity

c_b = 500 # backorder cost

In [170]:
obj_expr = []
for t in range(T-1):
    obj_expr += [c[t]*x[t] + h[t]*s[t] + c_b*b[t]]
obj_expr += [c[T-1]*x[T-1]]
obj = cp.Minimize(sum(obj_expr))

In [171]:
constraints = [
    x >= 0, s >= 0, x <= u, s <= w, b >= 0,
    x[0] + b[0] == d[0] + s[0],
    x[1] + s[0] + b[1] == d[1] + s[1] + b[0],
    x[2] + s[1] + b[2] == d[2] + s[2] + b[1],
    s[2] + x[3] == d[3] + b[2]
]

In [172]:
prob = cp.Problem(obj,constraints)
prob.solve(solver=cp.GLPK)

if (prob.status == 'optimal'):
    print('obj = ' + str(prob.value))
    print('x = ' + str(x.value))
    print('s = ' + str(s.value))
    print('b = ' + str(b.value))

obj = 44100.0
x = [100.   6.  54.  90.]
s = [30.  0.  0.]
b = [ 0. 14.  0.]


### Assignment problem

$$
\begin{array}{rll}
\mbox{max} & \sum_{i=1}^I \sum_{j=1}^J w_{ij} x_{ij}\\
\mbox{s.t.} & \sum_{i=1}^I x_{ij} \leq 1, & j = 1, \ldots, J,\\
& \sum_{j=1}^J x_{ij} \leq 1, & i = 1, \ldots, I,\\
& x_{ij} \geq 0, & i = 1, \ldots, I, j = 1, \ldots, J.
\end{array}
$$

In [213]:
# Data
I = 100
J = 200

# randomly generate the w matrix
np.random.seed(0)
w = np.array([[np.random.rand() for j in range(J)] for i in range(I)])

In [214]:
x = cp.Variable((I,J))


In [223]:
# define the objective expression
obj_expr = []
for i in range(I):
    for j in range(J):
        obj_expr += [w[i,j]*x[i,j]]
obj = cp.Maximize(sum(obj_expr))

constraints = []
for i in range(I):
    constraints += [sum([x[i,j] for j in range(J)]) <= 1]
    
for j in range(J):
    constraints += [sum([x[i,j] for i in range(I)]) <= 1]
    
constraints += [x >=0]

In [224]:
prob = cp.Problem(obj,constraint)
prob.solve(solver=cp.GLPK)

99.42296749780098